In [14]:
import requests
from bs4 import BeautifulSoup as bs
import time
import csv
from urllib.parse import urljoin
 
def get_car_links(main_url, page_number):
    try:
        url = f"{main_url}?p={page_number}"
        response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        
        if response.status_code == 429:
            print(f"Trop de requêtes, en pause pendant un moment...")
            time.sleep(2)  # Pause de 10 secondes en cas d'erreur 429
            return get_car_links(main_url, page_number)  # Réessayer après la pause
        
        response.raise_for_status()

        soup = bs(response.text, 'html.parser')
        base_url = "https://www.avito.ru/all/avtomobili"  # Ajoutez le domaine de base ici
        car_links = [urljoin(base_url, a['href']) for a in soup.select('a.styles-module-root-QmppR styles-module-root_noVisited-aFA10')]
        return car_links
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des liens de la page {page_number} : {e}")
        return []

def scrape_car_details(car_url):
    try:
        response = requests.get(car_url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        car_soup = bs(response.text, 'html.parser')

        # Obtenez le nom de la voiture
        car_name_element = car_soup.select_one('.styles-module-root-TWVKW.styles-module-root-_KFFt.styles-module-size_xxxl-A2qfi.styles-module-size_xxxl-_bK04.stylesMarningNormal-module-root-OSCNq.stylesMarningNormal-module-header-3xl-k0ckc')
        car_name = car_name_element.get_text(strip=True) if car_name_element else "Nom non trouvé"

        # Obtenez les détails de la voiture
        car_details_elements = car_soup.select('.item-view .item-params')  # Mise à jour du sélecteur
        car_features = {}

        for detail_element in car_details_elements:
            feature_icon = detail_element.select_one('.icones')
            feature_name = detail_element.select_one('p').get_text(strip=True) if detail_element.select_one('p') else "Nom de la caractéristique non trouvé"

            if feature_icon:
                feature_class = feature_icon.get('class')[1]
                car_features[feature_class] = feature_name

        # Ajoutez du code pour extraire le prix de la voiture
        car_price_element = car_soup.select_one('.styles-module-size_xxxl-A2qfi')
        car_price = car_price_element.get_text(strip=True) if car_price_element else "Prix non trouvé"

        print(f"Nom de la voiture: {car_name}")
        print(f"Caractéristiques de la voiture:\n{car_features}")
        print(f"Prix de la voiture: {car_price}\n{'='*30}")

        return {'Car Name': car_name, 'Car Features': car_features, 'Car Price': car_price}

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des détails de la voiture : {e}")
        return None

total_pages = 2  # Modifier le nombre total de pages selon les besoins
car_details_list = []

for page_number in range(1, total_pages + 1):
    main_page_url = 'https://www.avito.ru/all/avtomobili'
    car_links = get_car_links(main_page_url, page_number)

    for car_link in car_links:
        print(f"Scraping details for car: {car_link}")
        car_detail = scrape_car_details(car_link)
        
        if car_detail:
            car_details_list.append(car_detail)
        else:
            print(f"Aucun détail trouvé pour la voiture : {car_link}")

        time.sleep(2)

# Ajout de débogage
print("Nombre total de détails récupérés :", len(car_details_list))

# Écrire les détails des voitures dans un fichier CSV
with open('car_details_avito.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Car Name', 'Car Features', 'Car Price']  # Ajoutez 'Car Price' à la liste
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    for detail in car_details_list:
        csv_writer.writerow(detail)


Trop de requêtes, en pause pendant un moment...


KeyboardInterrupt: 

In [39]:
import requests
from bs4 import BeautifulSoup as bs
import time
import csv

def get_car_links(main_url, page_number):
    try:
        url = f"{main_url}?p={page_number}"
        response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = bs(response.text, 'html.parser')
        car_links = [a['href'] for a in soup.select('a._blank')]
        return car_links
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des liens de la page {page_number} : {e}")
        return []

import requests
from bs4 import BeautifulSoup

def scrape_car_details(url):
    try:
        # Faites une demande HTTP à l'URL de la voiture
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()

        # Analysez la page avec BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Obtenez le nom du véhicule
        name_element = soup.select_one('.block-title-list h2 a span.title-block.brand')
        car_name = name_element.text.strip() if name_element else "Nom non trouvé"

        # Obtenez le prix du véhicule
        price_element = soup.select_one('.prix')
        car_price = price_element.text.strip() if price_element else "Prix non trouvé"

        # Affichez les détails récupérés
        print(f"Nom du véhicule: {car_name}")
        print(f"Prix du véhicule: {car_price}")
        print('=' * 30)

        return {
            'Car Name': car_name,
            'Car Price': car_price
        }

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des détails du véhicule : {e}")
        return None


def scrap_pages(main_url, start_page, end_page):
    car_details_list = []

    for page_number in range(start_page, end_page + 1):
        car_links = get_car_links(main_url, page_number)

        for car_link in car_links:
            print(f"Scrapping details for car: {car_link}")
            car_detail = scrape_car_details(car_link)
            
            if car_detail:
                car_details_list.append(car_detail)
            else:
                print(f"Aucun détail trouvé pour la voiture : {car_link}")
            
            time.sleep(2)  # Respectez les politiques du site pour éviter d'être bloqué

    return car_details_list

main_page_url = 'https://www.avito.ru/all/avtomobili'
total_pages = 2  # Modifier le nombre total de pages selon les besoins
start_page = 1
end_page = total_pages
car_details_list = scrap_pages(main_page_url, start_page, end_page)

# Ajout de débogage
print("Nombre total de détails récupérés :", len(car_details_list))

# Écrire les détails des voitures dans un fichier CSV
with open('car_details_avito.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Car Name', 'Car Details', 'Car Price']  # Ajoutez 'Car Name' à la liste
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    for detail in car_details_list:
        csv_writer.writerow(detail)


Erreur lors de la récupération des liens de la page 1 : 429 Client Error: Too Many Requests for url: https://www.avito.ru/all/avtomobili?p=1
Nombre total de détails récupérés : 0


In [41]:
import requests
from bs4 import BeautifulSoup as bs
import time
import csv

def get_car_links(main_url, page_number):
    try:
        url = f"{main_url}?o={page_number}"
        response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = bs(response.text, 'html.parser')
        car_links = [a['href'] for a in soup.select('a.tracker.LeadLink')]
        return car_links
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des liens de la page {page_number} : {e}")
        return []

def scrape_car_details(car_url):
    try:
        response = requests.get(car_url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()

        car_soup = bs(response.text, 'html.parser')

        # Ajoutez cette ligne pour imprimer le HTML de la page
        print(car_soup.prettify())

        # Reste du code pour extraire les détails de la voiture
        brand = car_soup.select_one('.title-block.brand').text.strip()
        model = car_soup.select_one('.title-block.sub-title').text.strip()
        version = car_soup.select_one('.title-block.sub-title:nth-of-type(2)').text.strip()

        old_price_element = car_soup.select_one('.old-prix s')
        old_price = old_price_element.text.strip() if old_price_element else "Prix initial non trouvé"

        current_price_element = car_soup.select_one('.prix')
        current_price = current_price_element.text.strip() if current_price_element else "Prix actuel non trouvé"

        credit_price_element = car_soup.select_one('.lead-credit-price .credit-prix')
        credit_price = credit_price_element.text.strip() if credit_price_element else "Détails de crédit non trouvés"

        print(f"Marque: {brand}")
        print(f"Modèle: {model}")
        print(f"Version: {version}")
        print(f"Prix initial: {old_price}")
        print(f"Prix actuel: {current_price}")
        print(f"Détails de crédit: {credit_price}")
        print('=' * 30)

        return {
            'Brand': brand,
            'Model': model,
            'Version': version,
            'Old Price': old_price,
            'Current Price': current_price,
            'Credit Details': credit_price
        }

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des détails de la voiture : {e}")
        return None

main_page_url = 'https://www.leparking.fr/voiture-occasion/.html#!/voiture-occasion/%3Fid_pays%3D36'
total_pages = 2  # Modifier le nombre total de pages selon les besoins
car_details_list = []

for page_number in range(1, total_pages + 1):
    car_links = get_car_links(main_page_url, page_number)

    for car_link in car_links:
        print(f"Scrapping details for car: {car_link}")
        car_detail = scrape_car_details(car_link)
        
        if car_detail:
            car_details_list.append(car_detail)
        else:
            print(f"Aucun détail trouvé pour la voiture : {car_link}")
        
        time.sleep(2)  # Respect the website's policies to avoid being blocked

# Debugging information
print("Nombre total de détails récupérés :", len(car_details_list))

# Write car details to a CSV file
with open('car_details_ru.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Brand', 'Model', 'Version', 'Old Price', 'Current Price', 'Credit Details']
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    for detail in car_details_list:
        csv_writer.writerow(detail)


Nombre total de détails récupérés : 0


In [2]:
import requests
from bs4 import BeautifulSoup
import csv


def get_all_car_links(main_url):
    try:
        # Faites une demande HTTP à l'URL de la page principale
        response = requests.get(main_url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()

        # Analysez la page avec BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Obtenez tous les liens vers les voitures sur la page
        car_links = [a['href'] for a in soup.select('.block-title-list a[href^="/voiture-occasion-detail/"]')]

        return car_links

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des liens de voitures : {e}")
        return []

def scrape_all_cars(main_url):
    all_car_details = []
    current_page = 1

    while True:
        page_url = f"{main_url}?o={current_page}"
        car_links = get_all_car_links(page_url)

        if not car_links:
            print(f"Aucun lien de voiture trouvé sur la page {current_page}. Fin du scraping.")
            break

        for car_link in car_links:
            car_details = scrape_car_details(f"https://www.leparking.fr{car_link}")
            if car_details:
                all_car_details.append(car_details)

        current_page += 1

    return all_car_details

def scrape_car_details(url):
    try:
        # Faites une demande HTTP à l'URL de la voiture
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()

        # Analysez la page avec BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Obtenez le nom du véhicule
        name_element = soup.select_one('a.external.tag_f_titre')
        car_name = name_element.find('span', class_='title-block brand three-dots').text.strip() if name_element else "Nom non trouvé"

        # Obtenez le prix du véhicule
        price_element = soup.select_one('.prix')
        car_price = price_element.text.strip() if price_element else "Prix non trouvé"

        # Affichez les détails récupérés
        print(f"Nom du véhicule: {car_name}")
        print(f"Prix du véhicule: {car_price}")
        print('=' * 30)

        return {
            'Car Name': car_name,
            'Car Price': car_price
        }

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des détails du véhicule : {e}")
        return None

# Exemple d'utilisation pour une page principale
main_page_url = 'https://www.leparking.fr/voiture-occasion/.html#!/voiture-occasion/%3Fid_pays%3D36'
all_car_details = scrape_all_cars(main_page_url)

# Si des détails ont été récupérés avec succès, vous pouvez les utiliser comme bon vous semble
if all_car_details:
    print("Détails de toutes les voitures récupérés avec succès:")
    print(all_car_details)
else:
    print("Aucun détail trouvé pour les voitures.")

def save_to_csv(car_details, filename='car_details.csv'):
    fieldnames = ['Car Name', 'Car Price']

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()

        # Write the data
        for car in car_details:
            writer.writerow(car)


Nom du véhicule: Nom non trouvé
Prix du véhicule: 15 300 €
Nom du véhicule: Nom non trouvé
Prix du véhicule: 8 200 €
Nom du véhicule: Nom non trouvé
Prix du véhicule: 8 200 €
Nom du véhicule: Nom non trouvé
Prix du véhicule: 24 900 €
Nom du véhicule: Nom non trouvé
Prix du véhicule: 21 500 €
Nom du véhicule: Nom non trouvé
Prix du véhicule: 16 900 €
Nom du véhicule: Nom non trouvé
Prix du véhicule: 29 500 €


KeyboardInterrupt: 